In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('feature_matrix.csv')
ages = pd.read_csv('ages.csv')
X = df.copy(); y = ages.copy()
X.shape, y.shape

((22, 70), (22, 1))

### Linear Regression e Cross Validation

In a previous analysis (`baseline_models.ipynb`), I split the dataset into training and test dataset. This might not be the best approach to such a small dataset. I'll try with cross_validation here.

In [135]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate, train_test_split, GridSearchCV

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [43]:
lr_model = LinearRegression(fit_intercept=True)

In [44]:
# Define multiple scoring metrics
scoring = ['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2']

# perform cross validation
lr_results = cross_validate(lr_model, X=X, y=y, cv=7, scoring=scoring, return_train_score=False)

In [45]:
print(lr_results['test_neg_mean_squared_error'])  # MSE scores for each fold
print(lr_results['test_neg_mean_absolute_error']) # MAE scores for each fold
print(lr_results['test_r2'])                      # R^2 scores for each fold

[ -926.11349826 -4931.49188404 -1963.05008562 -1164.71443892
  -306.59554365 -1919.60278597 -2707.35378651]
[-21.62830585 -57.1644064  -39.3085158  -31.06624168 -16.20662173
 -35.09395314 -47.73411474]
[-17.60089595  -9.52773739  -4.39650148  -5.86427841  -9.99420158
 -22.68743376  -4.07753806]


The results vary quite a bit frome one fold to another. If we average the fold-scores, the model will be performing worse than the single train / test split model I tried before.

Some folds in cross-validation might contain patterns or outliers not present in others because the data points vary significantly from one another (heterogenous data). This is reflected in the results as we can see higher variance in model performance across folds, which was not as evident in a single train/test split in my previous exploration. 

In [46]:
print(f"mean MSE: {lr_results['test_neg_mean_squared_error'].mean()}")  # MSE scores for each fold
print(f"mean MAE: {lr_results['test_neg_mean_absolute_error'].mean()}") # MAE scores for each fold
print(f"mean R2: {lr_results['test_r2'].mean()}")                      # R^2 scores for each fold

mean MSE: -1988.4174318518558
mean MAE: -35.4574513350212
mean R2: -10.592655232137389


### Feature Selection

I'll try to use Feature Improtance from Random Forests to select a subset of the most informative features to reduce dimensionality and improve model performance

In [125]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [126]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train.values[:, 0])

RandomForestRegressor(random_state=42)

In [127]:
def evaluate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"R2: {r2}")

    return mae, mse, r2

In [128]:
y_pred = rf.predict(X_test)
evaluate_metrics(y_test, y_pred)

MAE: 16.155670333333337
MSE: 453.90340326152966
R2: -0.4190356869184484


(16.155670333333337, 453.90340326152966, -0.4190356869184484)

In [129]:
importances = rf.feature_importances_
feature_labels = X.columns
feature_importance_df = pd.DataFrame({'Feature':feature_labels, 'Importance':importances})
feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False)
feature_importance_df.head()

,Feature,Importance
46,Beta_RelPower_C_EC,0.326172
16,High_RelPower_T_EO,0.108954
41,DFV_F_EC,0.080194
11,Beta_RelPower_C_EO,0.078161
9,High_RelPower_C_EO,0.041281


In [130]:
sorted_indices = np.argsort(importances)[::-1] #to get in descneding order

In [131]:
k = 30
top_k_features = sorted_indices[ :k]

In [132]:
# select feature columns
X_train_selected = X_train.iloc[:, top_k_features] 
X_test_selected = X_test.iloc[:, top_k_features] 

In [133]:
# retrain and test
rf_selected = RandomForestRegressor(n_estimators=100, random_state=42)
rf_selected.fit(X_train_selected, y_train.values[:, 0])

RandomForestRegressor(random_state=42)

In [134]:
y_pred_selected = rf_selected.predict(X_test_selected)
evaluate_metrics(y_test, y_pred_selected)

MAE: 15.517421666666667
MSE: 454.478092627407
R2: -0.4208323350889902


(15.517421666666667, 454.478092627407, -0.4208323350889902)

We get a slightly better MAE with the top 30 features. 

### Hyperparameter Tuning with Grid Search

perform hyperparameter tuning for the Random Forest model using grid search

In [136]:
# Define the parameter grid for grid search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [138]:
rf = RandomForestRegressor(random_state=42)

In [140]:
# perform grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train.values[:, 0])

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_absolute_error')

In [144]:
# get the best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

In [145]:
# Evaluate the best model on the test set
y_pred_best = best_model.predict(X_test)
evaluate_metrics(y_test, y_pred_best)

MAE: 15.694491000000008
MSE: 431.2187945745286
R2: -0.34811692085648227


(15.694491000000008, 431.2187945745286, -0.34811692085648227)

R2 has improved. Others, not much

### Gradient Boosting

In [146]:
from sklearn.ensemble import GradientBoostingRegressor

In [147]:
gbm = GradientBoostingRegressor(random_state=42)

In [148]:
# Define the parameter grid for GBM
gbm_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [150]:
# Perform grid search with cross-validation for GBM
gbm_grid_search = GridSearchCV(estimator=gbm, param_grid=gbm_param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
gbm_grid_search.fit(X_train, y_train.values[:, 0])

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.3],
                         'max_depth': [3, 5, 7], 'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.8, 1.0]},
             scoring='neg_mean_absolute_error')

In [152]:
# Get the best GBM model
best_gbm = gbm_grid_search.best_estimator_

In [153]:
# Evaluate the best GBM model on the test set
gbm_y_pred = best_gbm.predict(X_test)
evaluate_metrics(y_test, gbm_y_pred)

MAE: 20.34637886627117
MSE: 667.5076536459583
R2: -1.086825467728532


(20.34637886627117, 667.5076536459583, -1.086825467728532)